# Import packages

In [19]:
import os
import json
import re
from sympy import simplify, symbols, collect


# import tensorflow as tf
# from tensorflow.keras.optimizers import Adam, RMSprop, SGD
# from tensorflow.keras import layers

# # Set random seeds for reproducibility
# seed_value = 2023
# tf.random.set_seed(seed_value)

# Constants and parameters

In [20]:
os.getcwd()

'/media/jesus/GSUS-DATA/CANADA_docs/RaymondSpiteri_USASK_SK/RKTK_equation_learning/nn-non-linear-systems'

In [28]:
EQS_FILE = 'ERK_equations_s3p3.json'

EQS_PATH = os.path.join(os.getcwd(), 'RK_rootedtrees', EQS_FILE)

# Load Equations

In [49]:
# Read the JSON data from the file
with open(EQS_PATH, 'r') as file:
    data = json.load(file)

variables = data['variables']
equations = data['equations']
equation_terms = data['equation_terms']

# display(
#     f'variables = {variables}',
#     f'equations = {equations}',
#     f'equation_terms = {equation_terms}'
# )

print(f'equation_terms = {equation_terms}')

# Initialize a list to store the coefficients
coefficients = []
functions = []

# Loop through the equation terms
for terms in equation_terms:
    equation_coeffs = [int(terms[0])]
    equation_functions = list()
    for term in terms[1:]:
        # Split each term by the first '*'
        term_parts = term.split('*')
        # print(f'term_parts = {term_parts}')
        # Extract the scalar coefficient or default to '1'
        scalar_coeff = term_parts[0] if len(term_parts) > 1 else '1'
        equation_coeffs.append(int(scalar_coeff))
        
        func_coeff = term_parts[-1] if len(term_parts) > 1 else term_parts[0]
        equation_functions.append(func_coeff)
        
        # # Convert the first part (scalar coefficient) to an integer
        # scalar_coeff = int(term_parts[0])
        # equation_coeffs.append(scalar_coeff)
        # # Append the remaining parts (terms with variables)
        # equation_coeffs.extend(term_parts[1:])
    coefficients.append(equation_coeffs)
    functions.append(equation_functions)

print(coefficients)
print(functions)

equation_terms = [['-1', 'b_1', 'b_2', 'b_3'], ['-1', '2*a_21*b_2', '2*b_3*(a_31 + a_32)'], ['-1', '6*a_21*a_32*b_3'], ['-1', '3*a_21^2*b_2', '3*b_3*(a_31 + a_32)^2']]
[[-1, 1, 1, 1], [-1, 2, 2], [-1, 6], [-1, 3, 3]]
[['b_1', 'b_2', 'b_3'], ['b_2', '(a_31 + a_32)'], ['b_3'], ['b_2', '(a_31 + a_32)^2']]


# Model

## Create model